<a href="https://colab.research.google.com/github/AmoghTantradi/Bot-Code/blob/master/12_lead_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wfdb
!pip install tensorflow
!pip install autokeras


     |████████████████████████████████| 122kB 7.7MB/s 
     |████████████████████████████████| 174kB 10.4MB/s 
     |████████████████████████████████| 71kB 6.2MB/s 


In [ ]:
import wfdb
from wfdb import io, plot
from collections import Counter
import os, glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import autokeras as ak
import tensorflow as tf
import shutil


In [ ]:
!gsutil -m cp -r gs://ptb-xl-1.0.1.physionet.org .
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
shutil.move('/content/drive/MyDrive/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1.zip','/content/')

!unzip /content/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1.zip 

os.rename('/content/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1', 'dataset')

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
from keras.applications import ResNet50
from keras.applications import VGG16
from keras.applications import VGG19
from keras.applications import DenseNet169


In [ ]:
import ast

keys=[]

def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    data = np.array([signal for signal, meta in data])
    return data

path = '/content/dataset/'
sampling_rate=100

# load and convert annotation data
Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

# Load raw signal data
X = load_raw_data(Y, sampling_rate, path)

# Load scp_statements.csv for diagnostic aggregation
agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]

def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
      if key=='NORM' or 'MI' in key:
        if key in agg_df.index:
            keys.append(key)
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

# Apply diagnostic superclass
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)

# Split data into train and test
test_fold = 10
# # Train
# trainX = X[np.where(Y.strat_fold != test_fold)]
# trainY = Y[((Y.strat_fold != test_fold))].diagnostic_superclass
# # Test
# testX = X[np.where(Y.strat_fold == test_fold)]
# testY = Y[Y.strat_fold == test_fold].diagnostic_superclass

trainX = X[:19634]
trainY = Y[:19364].diagnostic_superclass
# Test
testX = X[19634:]
testY = Y[19634:].diagnostic_superclass

In [ ]:
X.shape, Y.shape

In [ ]:
trainX.shape, trainY.shape, testX.shape, testY.shape

In [ ]:
Y[:5]

In [ ]:
pd.concat((pd.DataFrame(trainX.reshape(19634, 12000)), pd.Series(trainY)), axis=1)

In [ ]:
Counter(Y.strat_fold)

In [ ]:
Counter(keys)

In [ ]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

In [ ]:
y = []
for item in trainY:
  if len(item)==1:
    y.append(item[0])
Counter(y)

In [ ]:
Counter([len(item) for item in trainY])

In [ ]:
y_train=[]; train_inds=[]

for item in trainY:
  if len(item)==1:
    train_inds.append(list(trainY).index(item))
    y_train.append(item[0])

y_test=[]; test_inds=[]

for item in testY:
  if len(item)==1:
    test_inds.append(list(testY).index(item))
    y_test.append(item[0])


In [ ]:
record = wfdb.rdrecord('/content/dataset/records100/00000/00001_lr')
wfdb.plot_wfdb(record)

In [ ]:
plt.imshow(pd.DataFrame(X_train[0]).corr(),cmap='hot',interpolation='nearest')

In [ ]:
X_train = (trainX[train_inds]); X_test = testX[test_inds]

In [ ]:
pd.concat((pd.DataFrame(trainX.reshape(19634, 12000)), pd.Series(trainY)), axis=0)

In [ ]:
trainY.shape

In [ ]:
pd.DataFrame(X_train[0]).corr()

In [ ]:
X_train = (trainX[train_inds].reshape(13220, 80, 50, 3))
X_test = (testX[test_inds].reshape(1431, 80, 50, 3))
y_train = np.array([[1,0] if x == 'NORM' else [0,1] for x in y_train])
y_test = np.array([[1,0] if x == 'NORM' else [0,1] for x in y_test])

In [ ]:
testX[test_inds].shape

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


In [ ]:
X_train

In [ ]:
Counter([0 if x == 'NORM' else 1 for x in y_train])

In [ ]:
y

In [ ]:
y_train

In [ ]:

model = Sequential()

model.add(DenseNet169(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    input_shape = (80, 50, 3)
    ))

model.add(Dense(2, activation='sigmoid'))
print(model.layers)
model.layers[0].trainable = False

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# model.summary()

fit_history = model.fit(
    X_train[:4000], y_train[:4000],
    epochs=1,
    verbose=1,
    #callbacks=callbacks
)





In [ ]:
predictions = (np.argmax(model.predict(X_test),axis=1))

In [ ]:
Counter(trainY)

In [ ]:
y_hat = np.array([0 if x == 'NORM' else 1 for x in y_test])

In [ ]:
Counter(predictions)

In [ ]:
np.mean(predictions==y_hat)

In [ ]:
from tqdm import tqdm

In [ ]:
for record in tqdm(range(len(X_train[1:]))):
  channel0 = np.array((pd.DataFrame(pd.DataFrame(X_train[record])[0])))
  channel0s = np.hstack((channel0s, channel0))

In [ ]:
pd.DataFrame(channel0s)